Investigating Ulam Sequences
---

For two positive integers $a$ and $b$, the Ulam sequence is defined by
$$
\left\{
\begin{array}{rll}
U(a,b)_1 &= a \\
U(a,b)_2 &= b \\
U(a,b)_{k>2} &= \text{The smallest integer greater than }U(a,b)_{k-1}\text{ writable as } U(a,b)_i + U(a,b)_j, \text{ with }i\neq j
\end{array}
\right.
$$


Find
$$
\sum_{n=2}^{10} U(2, 2n+1)_k,
$$
where $k = 10^{11}$.

In [216]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

ulam_parameters = [
       [a,b,k]
    for a in range(1,5)
    for b in range(1,13)
    for k in range(1,51)
]

ulam_columns = ['a','b','k']
udf = pd.DataFrame(ulam_parameters, columns = ulam_columns)
udf['ulam'] = np.nan
udf.loc[udf['k']==1,'ulam'] = udf.loc[udf['k']==1,'a']
udf.loc[udf['k']==2,'ulam'] = udf.loc[udf['k']==2, 'b']
udf.index.rename('index', inplace = True)
udf.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400 entries, 0 to 2399
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   a       2400 non-null   int64  
 1   b       2400 non-null   int64  
 2   k       2400 non-null   int64  
 3   ulam    96 non-null     float64
dtypes: float64(1), int64(3)
memory usage: 75.1 KB


In [2]:
udf.head()

,a,b,k,ulam
index,,,,
0,1,1,1,1.0
1,1,1,2,1.0
2,1,1,3,NaN
3,1,1,4,NaN
4,1,1,5,NaN


In [3]:
udf.describe()

,a,b,k,ulam
count,2400.000000,2400.000000,2400.000000,96.000000
mean,2.500000,6.500000,25.500000,4.500000
std,1.118267,3.452772,14.433877,3.270281
min,1.000000,1.000000,1.000000,1.000000
25%,1.750000,3.750000,13.000000,2.000000
50%,2.500000,6.500000,25.500000,3.500000
75%,3.250000,9.250000,38.000000,6.250000
max,4.000000,12.000000,50.000000,12.000000


In [4]:
udf[udf['ulam'].notna()]

,a,b,k,ulam
index,,,,
0,1,1,1,1.0
1,1,1,2,1.0
50,1,2,1,1.0
51,1,2,2,2.0
100,1,3,1,1.0
...,...,...,...,...
2251,4,10,2,10.0
2300,4,11,1,4.0
2301,4,11,2,11.0


Generate some of the smaller ulam numbers by brute force.

In [202]:
def get_ulam_from_df(a,b,k, df):
    """
    Blindly return the value stored in the dataframe's ulam given a,b, and k.
    Does not check if the value is valid, or null, or if the given collection of a,b,k are present.
    """
    ulam_val = df.loc[
        (df['a'] == a) & (df['b'] == b) & (df['k'] == k),
        'ulam'
    ]
    return ulam_val
    

def ulam(a, b, k, df, change_df_inplace = True):
    """
    Return the ulam number U(a,b)_k.
    First, it consults the dataframe for a value if it's already there.
    If not, it attempts to generate it.
    """
    #check for proper input
    if a<1 or b<1 or k<1 or type(a) != int or type(b) != int or type(k)!= int:
        raise(
            ValueError('Use positive integers for a, b, k.'))
    #check if the dataframe already has the value stored and return it if so
    dataframe_value = get_ulam_from_df(a,b,k,df)
    if dataframe_value.notna().iloc[0]:
        return int(dataframe_value)
    #if the dataframe doesn't already have the value, find it.
    else:
        #get the previous ulam number. the next one must be larger
        last_ulam_number = ulam(a, b, k-1, df)
        #get all the ulam numbers in the dataframe for a and b.
        this_previous_ulam_mask = (df['a'] == a) & (df['b'] == b) & df['ulam'].notna()
        all_previous_ulam_numbers = df.loc[this_previous_ulam_mask, 'ulam']
        # #start looking for the next candidate just above the last ulam number.
        # #this will increment in the loop below
        # this_ulam_candidate = last_ulam_number
        # searching_for_next_ulam_number = True
        # #find all previous ulam sums
        
        #create an offset if the first two ulam numbers are the same. They have to be treated
        #as distinct
        if a == b:
            offset = 1
        else:
            offset = 0
            
        all_previous_ulam_sums = np.array([
                all_previous_ulam_numbers.iloc[index1] + all_previous_ulam_numbers.iloc[index2]
                for index1 in range(offset + len(all_previous_ulam_numbers) - 1)
                for index2 in range(index1+1, len(all_previous_ulam_numbers))
            ])
        all_previous_ulam_sums = pd.DataFrame(all_previous_ulam_sums, columns = ['sum'])
        #drop any candidates that show up more than once.
        
        # return all_previous_ulam_sums
        all_previous_ulam_sums.drop_duplicates(keep = False, inplace = True)
        #drop any candidates that are not larger than the laster one.
        size_mask = all_previous_ulam_sums > last_ulam_number
        # all_previous_ulam_sums = all_previous_ulam_sums[size_mask]
        
        #get the smallest number with the size mask applied. This is the ulam number.
        ulam_number = int(all_previous_ulam_sums[size_mask].min())
        
        #write it into the df
        if change_df_inplace:
            df_mask = (df['a'] == a) & (df['b'] == b) & (df['k'] == k)
            df.loc[df_mask, 'ulam'] = ulam_number
            
        return ulam_number
        
        # return all_previous_ulam_sums
        # return all_previous_ulam_numbers
        
        #remove all ele
        
#         while searching_for_next_ulam_number:
#             this_ulam_candidate += 1
#             # print(f'{this_ulam_candidate}')
            
#             num_sums_are_this_candidate = (all_previous_ulam_sums == this_ulam_candidate).sum()
#             if num_sums_are_this_candidate == 1:
#                 searching_for_next_ulam_number != searching_for_next_ulam_number
#                 df.loc[this_previous_ulam_mask & (df['k'] == k), 'ulam'] = this_ulam_candidate
#             else:
#                 pass


    return this_ulam_candidate
    

In [203]:
ulam(2,2,8, udf)

22

In [204]:
ulam(1,2,1, udf)

1

In [205]:
ulam(1,2,2, udf)

2

In [206]:
ulam(1,2,3,udf)

3

In [207]:
ulam(1,2,4,udf)

4

In [208]:
ulam(1,2,5,udf)

6

In [209]:
ulam(1,2,6,udf)

8

In [213]:
ulam(1,2,35,udf)

155

In [214]:
udf[(udf['a'] == 1)& (udf['b'] == 2)]

,a,b,k,ulam
index,,,,
50,1,2,1,1.0
51,1,2,2,2.0
52,1,2,3,3.0
53,1,2,4,4.0
54,1,2,5,6.0
55,1,2,6,8.0
56,1,2,7,11.0
57,1,2,8,13.0
58,1,2,9,16.0


Great. Ulam numbers are being generated.

In [225]:
ulam(3,4,15,udf)

41

In [226]:
udf

,a,b,k,ulam
index,,,,
0,1,1,1,1.0
1,1,1,2,1.0
2,1,1,3,NaN
3,1,1,4,NaN
4,1,1,5,NaN
...,...,...,...,...
2395,4,12,46,NaN
2396,4,12,47,NaN
2397,4,12,48,NaN
